In [1]:
import cv2
import numpy as np
from typing import List
import matplotlib
import matplotlib.pyplot as plt
#image_file = r"C:\Users\cola\Downloads\test.jpg"
#img = cv2.imread(image_file)
#gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
#img_erode = cv2.erode(thresh, np.ones((3, 3), np.uint8), iterations=1)

print(cv2.__version__)


4.2.0


In [2]:
#contours, hierarchy = cv2.findContours(img_erode, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

In [3]:
#output = img.copy()

In [4]:
#for idx, contour in enumerate(contours):
  #  (x, y, w, h) = cv2.boundingRect(contour)
    # print("R", idx, x, y, w, h, cv2.contourArea(contour), hierarchy[0][idx])
    # hierarchy[i][0]: the index of the next contour of the same level
    # hierarchy[i][1]: the index of the previous contour of the same level
    # hierarchy[i][2]: the index of the first child
    # hierarchy[i][3]: the index of the parent
  #  if hierarchy[0][idx][3] == 0:
    #    cv2.rectangle(output, (x, y), (x + w, y + h), (70, 0, 0), 1)

In [5]:
#cv2.imshow("Input", img)
#cv2.imshow("Enlarged", img_erode)
#cv2.imshow("Output", output)
#cv2.waitKey(0)

In [6]:
def letters_extract(image_file: str, out_size=28) -> List[any]:
    img = cv2.imread(image_file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    img_erode = cv2.erode(thresh, np.ones((3, 3), np.uint8), iterations=1)
     # Get contours
    contours, hierarchy = cv2.findContours(img_erode, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    output = img.copy()
    letters = []
    for idx, contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        # print("R", idx, x, y, w, h, cv2.contourArea(contour), hierarchy[0][idx])
        # hierarchy[i][0]: the index of the next contour of the same level
        # hierarchy[i][1]: the index of the previous contour of the same level
        # hierarchy[i][2]: the index of the first child
        # hierarchy[i][3]: the index of the parent
        if hierarchy[0][idx][3] == 0:
            cv2.rectangle(output, (x, y), (x + w, y + h), (70, 0, 0), 1)
            letter_crop = gray[y:y + h, x:x + w]
            # print(letter_crop.shape)
            # Resize letter canvas to square
            size_max = max(w, h)
            letter_square = 255 * np.ones(shape=[size_max, size_max], dtype=np.uint8)
            if w > h:
                # Enlarge image top-bottom
                # ------
                # ======
                # ------
                y_pos = size_max//2 - h//2
                letter_square[y_pos:y_pos + h, 0:w] = letter_crop
            elif w < h:
                # Enlarge image left-right
                # --||--
                x_pos = size_max//2 - w//2
                letter_square[0:h, x_pos:x_pos + w] = letter_crop
            else:
                letter_square = letter_crop

            # Resize letter to 28x28 and add letter and its X-coordinate
            letters.append((x, w, cv2.resize(letter_square, (out_size, out_size), interpolation=cv2.INTER_AREA)))

    # Sort array in place by X-coordinate
    letters.sort(key=lambda x: x[0], reverse=False)

    return letters

In [7]:
#letters = letters_extract(image_file)
##cv2.imshow("0", letters[0][2])
#cv2.imshow("1", letters[1][2])
#cv2.imshow("2", letters[2][2])
#cv2.imshow("3", letters[3][2])
#cv2.imshow("4", letters[4][2])
#cv2.waitKey(0)

In [8]:
emnist_labels = [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122]

In [9]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Reshape, LSTM, BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras import backend as K
#from tensorflow.keras.constraints import maxnorm
import tensorflow as tf

In [10]:
def emnist_model():
    model = Sequential()
    model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='valid', input_shape=(28, 28, 1), activation='relu'))
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(emnist_labels), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    return model

In [11]:
!pip install --user idx2numpy
import idx2numpy
#import emnist



In [16]:
emnist_path = 'C:/Users/orlov/emnist/'
X_train = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-train-images-idx3-ubyte')
y_train = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-train-labels-idx1-ubyte')
plt.imshow(X_train[50])
print(y_train[50])


X_test = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-test-images-idx3-ubyte')
y_test = idx2numpy.convert_from_file(emnist_path + 'emnist-byclass-test-labels-idx1-ubyte')

X_train = np.reshape(X_train, (X_train.shape[0], 28, 28, 1))
X_test = np.reshape(X_test, (X_test.shape[0], 28, 28, 1))

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, len(emnist_labels))

k = 10
X_train = X_train[:X_train.shape[0] // k]
y_train = y_train[:y_train.shape[0] // k]
X_test = X_test[:X_test.shape[0] // k]
y_test = y_test[:y_test.shape[0] // k]

# Normalize
X_train = X_train.astype(np.float32)
X_train /= 255.0
X_test = X_test.astype(np.float32)
X_test /= 255.0

x_train_cat = keras.utils.to_categorical(y_train, len(emnist_labels))
y_test_cat = keras.utils.to_categorical(y_test, len(emnist_labels))
 
print X_train[0]

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(X_train[0])? (<ipython-input-16-2ca412442f1c>, line 31)

In [15]:
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

# Required for learning_rate_reduction:
#keras.backend.get_session().run(tf.global_variables_initializer())
s = tf.compat.v1.Session()
with tf.compat.v1.get_default_graph().as_default():
    h = tf.compat.v1.global_variables_initializer()

s.run(h)

model = emnist_model()
#tf.compat.v1.keras.backend.get_session().run(tf.compat.v1.global_variables_initializer())
model.fit(X_train, x_train_cat, validation_data=(X_test, y_test_cat), callbacks=[learning_rate_reduction], batch_size=64, epochs=60)

model.save('emnist_letters.h5')

Train on 69793 samples, validate on 11632 samples
Epoch 1/60
69793/69793 [==============================] - 94s 1ms/sample - loss: 4.0257 - accuracy: 0.0723 - val_loss: 3.8891 - val_accuracy: 0.2432
Epoch 2/60
69793/69793 [==============================] - 74s 1ms/sample - loss: 3.7610 - accuracy: 0.1466 - val_loss: 3.5656 - val_accuracy: 0.3319
Epoch 3/60
69793/69793 [==============================] - 72s 1ms/sample - loss: 3.4711 - accuracy: 0.2210 - val_loss: 3.2411 - val_accuracy: 0.3828
Epoch 4/60
69793/69793 [==============================] - 77s 1ms/sample - loss: 3.2009 - accuracy: 0.2909 - val_loss: 2.9339 - val_accuracy: 0.4249
Epoch 5/60
69793/69793 [==============================] - 77s 1ms/sample - loss: 2.9454 - accuracy: 0.3454 - val_loss: 2.6543 - val_accuracy: 0.4548
Epoch 6/60
69793/69793 [==============================] - 78s 1ms/sample - loss: 2.7202 - accuracy: 0.3873 - val_loss: 2.4232 - val_accuracy: 0.4795
Epoch 7/60
69793/69793 [==============================] 

69793/69793 [==============================] - 99s 1ms/sample - loss: 1.7008 - accuracy: 0.5632 - val_loss: 1.4332 - val_accuracy: 0.6244
Epoch 22/60
69793/69793 [==============================] - 99s 1ms/sample - loss: 1.6754 - accuracy: 0.5674 - val_loss: 1.4099 - val_accuracy: 0.6299
Epoch 23/60
69793/69793 [==============================] - 95s 1ms/sample - loss: 1.6510 - accuracy: 0.5739 - val_loss: 1.3882 - val_accuracy: 0.6338
Epoch 24/60
69793/69793 [==============================] - 100s 1ms/sample - loss: 1.6340 - accuracy: 0.5753 - val_loss: 1.3682 - val_accuracy: 0.6382
Epoch 25/60
69793/69793 [==============================] - 102s 1ms/sample - loss: 1.6098 - accuracy: 0.5814 - val_loss: 1.3495 - val_accuracy: 0.6419
Epoch 26/60
69793/69793 [==============================] - 99s 1ms/sample - loss: 1.6021 - accuracy: 0.5837 - val_loss: 1.3327 - val_accuracy: 0.6444
Epoch 27/60
69793/69793 [==============================] - 101s 1ms/sample - loss: 1.5823 - accuracy: 0.5853 -

69793/69793 [==============================] - 90s 1ms/sample - loss: 1.4038 - accuracy: 0.6202 - val_loss: 1.1450 - val_accuracy: 0.6862
Epoch 43/60
69793/69793 [==============================] - 91s 1ms/sample - loss: 1.3965 - accuracy: 0.6222 - val_loss: 1.1361 - val_accuracy: 0.6884
Epoch 44/60
69793/69793 [==============================] - 90s 1ms/sample - loss: 1.3929 - accuracy: 0.6226 - val_loss: 1.1288 - val_accuracy: 0.6905
Epoch 45/60
69793/69793 [==============================] - 91s 1ms/sample - loss: 1.3825 - accuracy: 0.6257 - val_loss: 1.1216 - val_accuracy: 0.6909
Epoch 46/60
69793/69793 [==============================] - 90s 1ms/sample - loss: 1.3722 - accuracy: 0.6272 - val_loss: 1.1142 - val_accuracy: 0.6930
Epoch 47/60
69793/69793 [==============================] - 90s 1ms/sample - loss: 1.3624 - accuracy: 0.6287 - val_loss: 1.1071 - val_accuracy: 0.6943
Epoch 48/60
69793/69793 [==============================] - 80s 1ms/sample - loss: 1.3558 - accuracy: 0.6297 - va

In [20]:
model = keras.models.load_model('emnist_letters.h5')

def emnist_predict_img(model, img):
    img_arr = np.expand_dims(img, axis=0)
    img_arr = 1 - img_arr/255.0
    #img_arr[0] = np.rot90(img_arr[0], 3)
    img_arr[0] = np.fliplr(img_arr[0])
    img_arr = img_arr.reshape((1, 28, 28, 1))

    result = model.predict_classes([img_arr])
    return chr(emnist_labels[result[0]])

In [21]:
def img_to_str(model, image_file):
    letters = letters_extract(image_file)
    s_out = ""
    for i in range(len(letters)):
        dn = letters[i+1][0] - letters[i][0] - letters[i][1] if i < len(letters) - 1 else 0
        s_out += emnist_predict_img(model, letters[i][2])
        if (dn > letters[i][1]/4):
            s_out += ' '
    return s_out

In [22]:
model = keras.models.load_model('emnist_letters.h5')

In [23]:
s_out = img_to_str(model, 'a.png')

In [24]:
print(s_out)

K
